Loads TensorFlow and necessary tools for:

Creating the model.

Preprocessing the images.

Training and testing the model.

In [15]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.preprocessing import image


Prepares training and testing data.

Resizes images to 150×150 and normalizes pixel values to be between 0 and 1.

Organizes images into batches of 32 for efficient processing.

In [17]:
train_dir = '/content/drive/My Drive/data/train'  # Path to training data folder
test_dir = '/content/drive/My Drive/data/test'    # Path to testing data folder

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 1048 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


Builds a Convolutional Neural Network (CNN) with:

3 convolutional layers for extracting image features.

3 pooling layers for reducing image size while retaining important information.

2 dense layers:

One with 128 neurons (hidden layer).

Another with 3 neurons for output (one for each class).

In [18]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Sets up the model for training:

Adam optimizer adjusts weights to reduce errors.

Crossentropy loss measures how well the model predicts the correct class.

Tracks accuracy during training.

In [19]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


What it does:

Trains the model for 10 epochs (10 complete passes through the training data).

Validates the model using the testing data.

In [20]:
epochs = 10
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs
)


Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.4911 - loss: 1.1159 - val_accuracy: 0.3041 - val_loss: 1.2433
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.7834 - loss: 0.5848 - val_accuracy: 0.3684 - val_loss: 1.4190
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.8551 - loss: 0.4300 - val_accuracy: 0.4094 - val_loss: 1.2746
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.8712 - loss: 0.3688 - val_accuracy: 0.5146 - val_loss: 1.3680
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9045 - loss: 0.2756 - val_accuracy: 0.4737 - val_loss: 1.5365
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.9215 - loss: 0.2403 - val_accuracy: 0.4152 - val_loss: 1.6855
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.9155 - loss: 0.2173 - val_accuracy: 0.3275 - val_loss: 1.7492
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.9376 - loss: 0.1667 - val_accuracy: 0.2924 - val_loss:

Loads an image from your computer.

Prepares the image for prediction (resize, normalize, etc.).

Uses the model to predict the class of the image.

Prints the predicted class and confidence level.

In [30]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Function to predict a single image
def predict_image(model, image_path):
    img = load_img(image_path, target_size=(150, 150))  # Load and resize the image
    img_array = img_to_array(img)                      # Convert to a NumPy array
    img_array = img_array / 255.0                      # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)      # Add batch dimension
    predictions = model.predict(img_array)            # Predict
    class_indices = train_generator.class_indices      # Get class labels
    class_labels = {v: k for k, v in class_indices.items()}  # Reverse mapping
    predicted_class = class_labels[np.argmax(predictions)]
    confidence = np.max(predictions)
    return predicted_class, confidence


from google.colab import files

# Function to upload and test an image in Colab
def upload_and_test():
    print("Please upload an image:")
    uploaded = files.upload()  # This opens a file upload dialog
    for filename in uploaded.keys():
        print(f"File {filename} uploaded.")
        predicted_class, confidence = predict_image(model, filename)
        print(f"Predicted Class: {predicted_class}, Confidence: {confidence:.2f}")


upload_and_test()



Please upload an image:


Saving 2.759728-fire-in-ipcl-plant-113018.jpg to 2.759728-fire-in-ipcl-plant-113018.jpg
File 2.759728-fire-in-ipcl-plant-113018.jpg uploaded.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Class: fire, Confidence: 1.00
